In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

37 documents
11 classes ['file', 'goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today', 'tomorrow']
58 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'c', 'can', 'card', 'cash', 'credit', 'd', 'day', 'do', 'doe', 'find', 'good', 'goodby', 'hav', 'hello', 'help', 'her', 'hi', 'hom', 'hour', 'how', 'i', 'ident', 'is', 'kind', 'lat', 'lik', 'loc', 'mastercard', 'mop', 'nextday', 'of', 'on', 'op', 'rent', 'search', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'tomorrow', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [4]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

In [5]:
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model4.tflearn')

Training Step: 4999  | total loss: 0.01659 | time: 0.018s
| Adam | epoch: 1000 | loss: 0.01659 - acc: 1.0000 -- iter: 32/37
Training Step: 5000  | total loss: 0.01658 | time: 0.022s
| Adam | epoch: 1000 | loss: 0.01658 - acc: 1.0000 -- iter: 37/37
--
INFO:tensorflow:/Users/harishankar/Documents/Work/Chennai/ChatBot/model4.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))


In [7]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [8]:
# restore all of our data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [9]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

In [10]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [11]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0]
['file', 'goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today', 'tomorrow']


In [18]:
# load our saved model
model.load('./model4.tflearn')

INFO:tensorflow:Restoring parameters from /Users/harishankar/Documents/Work/Chennai/ChatBot/model4.tflearn


NotFoundError: Key Accuracy/Mean/moving_avg_1 not found in checkpoint
	 [[Node: save_5/RestoreV2_1 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_5/Const_0_0, save_5/RestoreV2_1/tensor_names, save_5/RestoreV2_1/shape_and_slices)]]

Caused by op 'save_5/RestoreV2_1', defined at:
  File "/Users/harishankar/anaconda/envs/tfdeeplearning/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/harishankar/anaconda/envs/tfdeeplearning/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/harishankar/anaconda/envs/tfdeeplearning/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/harishankar/anaconda/envs/tfdeeplearning/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/harishankar/anaconda/envs/tfdeeplearning/lib/python3.5/site-packages/i

In [15]:
ls

03-TF-Neural-Network.ipynb*
05-Crash-Course-Review-Exercises-Solutions.ipynb*
05-Crash-Course-Review-Exercises.ipynb*
Hello TF.ipynb
Rx_training_data
Rxmodel.tflearn.data-00000-of-00001
Rxmodel.tflearn.index
Rxmodel.tflearn.meta
Untitled.ipynb
Untitled1.ipynb
Untitled10.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
Untitled6.ipynb
Untitled7.ipynb
Untitled8.ipynb
Untitled9.ipynb
checkpoint
data_viz.ipynb
intents.json
mac_tfdl_env.yml
model.tflearn.data-00000-of-00001
model.tflearn.index
model.tflearn.meta
model2.tflearn.data-00000-of-00001
model2.tflearn.index
model2.tflearn.meta
model3.tflearn.data-00000-of-00001
model3.tflearn.index
model3.tflearn.meta
model4.tflearn.data-00000-of-00001
model4.tflearn.index
model4.tflearn.meta
tflearn_logs/
training_data


In [16]:
cd ./

/Users/harishankar/Documents/Work/Chennai/ChatBot


In [17]:
ls

03-TF-Neural-Network.ipynb*
05-Crash-Course-Review-Exercises-Solutions.ipynb*
05-Crash-Course-Review-Exercises.ipynb*
Hello TF.ipynb
Rx_training_data
Rxmodel.tflearn.data-00000-of-00001
Rxmodel.tflearn.index
Rxmodel.tflearn.meta
Untitled.ipynb
Untitled1.ipynb
Untitled10.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
Untitled6.ipynb
Untitled7.ipynb
Untitled8.ipynb
Untitled9.ipynb
checkpoint
data_viz.ipynb
intents.json
mac_tfdl_env.yml
model.tflearn.data-00000-of-00001
model.tflearn.index
model.tflearn.meta
model2.tflearn.data-00000-of-00001
model2.tflearn.index
model2.tflearn.meta
model3.tflearn.data-00000-of-00001
model3.tflearn.index
model3.tflearn.meta
model4.tflearn.data-00000-of-00001
model4.tflearn.index
model4.tflearn.meta
tflearn_logs/
training_data


In [ ]:
grep 